In [1]:
import gensim
import os
import collections
import smart_open
import random

Using TensorFlow backend.



Getting Started

To get going, we'll need to have a set of documents to train our doc2vec model. In theory, a document could be anything from a short 140 character tweet, a single paragraph (i.e., journal article abstract), a news article, or a book. In NLP parlance a collection or set of documents is often referred to as a corpus.

For this tutorial, we'll be training our model using the Lee Background Corpus included in gensim. This corpus contains 314 documents selected from the Australian Broadcasting Corporation’s news mail service, which provides text e-mails of headline stories and covers a number of broad topics.

And we'll test our model by eye using the much shorter Lee Corpus which contains 50 documents.


In [2]:
# Set file names for train and test data
test_data_dir = "../../data/"
lee_train_file = test_data_dir + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'


Define a Function to Read and Preprocess Text

Below, we define a function to open the train/test file (with latin encoding), read the file line-by-line, pre-process each line using a simple gensim pre-processing tool (i.e., tokenize text into individual words, remove punctuation, set to lowercase, etc), and return a list of words. Note that, for a given file (aka corpus), each continuous line constitutes a single document and the length of each line (i.e., document) can vary. Also, to train the model, we'll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply the zero-based line number.


In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [4]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [5]:
train_corpus[:1]


[TaggedDocument(words=[u'hundreds', u'of', u'people', u'have', u'been', u'forced', u'to', u'vacate', u'their', u'homes', u'in', u'the', u'southern', u'highlands', u'of', u'new', u'south', u'wales', u'as', u'strong', u'winds', u'today', u'pushed', u'huge', u'bushfire', u'towards', u'the', u'town', u'of', u'hill', u'top', u'new', u'blaze', u'near', u'goulburn', u'south', u'west', u'of', u'sydney', u'has', u'forced', u'the', u'closure', u'of', u'the', u'hume', u'highway', u'at', u'about', u'pm', u'aedt', u'marked', u'deterioration', u'in', u'the', u'weather', u'as', u'storm', u'cell', u'moved', u'east', u'across', u'the', u'blue', u'mountains', u'forced', u'authorities', u'to', u'make', u'decision', u'to', u'evacuate', u'people', u'from', u'homes', u'in', u'outlying', u'streets', u'at', u'hill', u'top', u'in', u'the', u'new', u'south', u'wales', u'southern', u'highlands', u'an', u'estimated', u'residents', u'have', u'left', u'their', u'homes', u'for', u'nearby', u'mittagong', u'the', u'ne

In [6]:
print(test_corpus[:1])

[[u'the', u'national', u'executive', u'of', u'the', u'strife', u'torn', u'democrats', u'last', u'night', u'appointed', u'little', u'known', u'west', u'australian', u'senator', u'brian', u'greig', u'as', u'interim', u'leader', u'shock', u'move', u'likely', u'to', u'provoke', u'further', u'conflict', u'between', u'the', u'party', u'senators', u'and', u'its', u'organisation', u'in', u'move', u'to', u'reassert', u'control', u'over', u'the', u'party', u'seven', u'senators', u'the', u'national', u'executive', u'last', u'night', u'rejected', u'aden', u'ridgeway', u'bid', u'to', u'become', u'interim', u'leader', u'in', u'favour', u'of', u'senator', u'greig', u'supporter', u'of', u'deposed', u'leader', u'natasha', u'stott', u'despoja', u'and', u'an', u'outspoken', u'gay', u'rights', u'activist']]




Notice that the testing corpus is just a list of lists and does not contain any tags.
Training the Model
Instantiate a Doc2Vec Object

Now, we'll instantiate a Doc2Vec model with a vector size with 50 words and iterating over the training corpus 10 times. We set the minimum word count to 2 in order to give higher frequency words more weighting. Model accuracy can be improved by increasing the number of iterations but this generally increases the training time.


In [7]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)


Build a Vocabulary

In [8]:
model.build_vocab(train_corpus)




Essentially, the vocabulary is a dictionary (accessible via model.vocab) of all of the unique words extracted from the training corpus along with the count (e.g., model.vocab['penalty'].count for counts for the word penalty).
Time to Train

If the BLAS library is being used, this should take no more than 2 seconds. If the BLAS library is not being used, this should take no more than 2 minutes, so use BLAS if you value your time.


In [9]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)


CPU times: user 7.54 s, sys: 444 ms, total: 7.99 s
Wall time: 3.27 s


2348114


Inferring a Vector

One important thing to note is that you can now infer a vector for any piece of text without having to re-train the model by passing a list of words to the model.infer_vector function. This vector can then be compared with other vectors via cosine similarity.


In [10]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])


array([-0.05338683, -0.05621618, -0.01968828, -0.01180828, -0.03748288,
        0.02934933, -0.00492503, -0.00208799, -0.057404  , -0.04342225,
       -0.0118835 , -0.09448473, -0.04083   ,  0.05746741,  0.06935102,
       -0.03003963, -0.0846075 ,  0.06261422,  0.01573074, -0.00782143,
        0.00297125, -0.00123506, -0.06259527,  0.01617383,  0.03104758,
        0.0710265 , -0.02191747,  0.0373035 ,  0.06481646, -0.02447152,
        0.00135802, -0.00331402,  0.04362392, -0.02822047, -0.01391614,
        0.03912758,  0.030517  , -0.03057335, -0.01065949,  0.01620738,
        0.01328738, -0.06407116, -0.0968592 ,  0.05417617,  0.04710863,
        0.04481058, -0.06332932, -0.0287804 ,  0.03569229,  0.01144832], dtype=float32)

In [11]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [12]:
collections.Counter(ranks)  # Results vary due to random seeding and very small corpus


Counter({0: 291, 1: 8, 2: 1})

In [13]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [14]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (177): «israeli helicopters have again attacked palestinian targets in the gaza strip it is the latest in series of attacks designed to put pressure on the palestinian authority to take tougher action against militants accused of violence against israelis two helicopters returned to the skies above gaza north in the early hours of tuesday morning eyewitnesses report seeing them fire as many as four missiles into security post in the north eastern township of bayt hanum the small one storey building is local command station for yasser arafat personal guard force the building was completely destroyed but nobody was injured in the attack israel has accused force of direct involvement in the preparation and execution of attacks against its people but the palestinian authority insists it is legitimate force and key part of the palestinian security infrastructure»

Similar Document (208, 0.7363927364349365): «israeli tanks and troops have launched two incursions in the gaza st

In [15]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (18): «in malawi as in other countries in the region aids is making the effects of the famine much worse the overall hiv infection rate in malawi is per cent but in some areas up to percent of people are infected significant proportion of the young adult population is too sick to do any productive work malnutrition causes people to succumb to the disease much more quickly than they do in the west and hunger forces women into prostitution in order to feed their families making them more vulnerable to contracting the disease life expectancy has been reduced to years»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (123, 0.43879228830337524): «new report has revealed there are fewer young people using homeless services than widely thought the australian institute of health and welfare report shows just over per cent of people aged between and used such services over the past year the main reason for young people seeking assistance was based on